# 01-Data Treatment

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from scipy import stats
%matplotlib inline
import warnings; warnings.simplefilter('ignore')

# Packages for this notebook
import random
from random import randint
import math
import datetime
from statistics import mean    
from numpy import array


# Multiple imputation packages
from impyute.imputation.cs import mice
from missingpy import MissForest

# Error metrics packages
from tsmetrics import tsmetrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import max_error
from sklearn.metrics import r2_score

---

## 1.3 Imputation Study

In [6]:
# Import dataset from 01-00-Data_Treatment notebook
dt = pd.read_csv('Preprocessed_Data/_01_dt_00.csv',index_col=[0],parse_dates=[0], header=0)

In [3]:
dt.head(3)

,civil,south_tower,wt_temp,wt_tmpap,wt_hr,wt_max_windgust,wt_mean_windspd,wt_mean_pres,wt_mean_solarrad,wt_rain_day,t_hour,t_month,t_dayofweek,t_year,miss_civil,miss_south_tower,miss_wt
timestamp,,,,,,,,,,,,,,,,,
2014-01-01 01:00:00,65.99413,114.897996,12.6,10.5,88.3,7.3,3.4,1021.0,0.6,4.9,1,1,2,2014,0,0,0
2014-01-01 02:00:00,66.20412,114.819072,12.9,10.3,87.3,7.8,4.0,1021.0,0.7,4.9,2,1,2,2014,0,0,0
2014-01-01 03:00:00,67.27316,114.491295,13.1,10.2,86.2,8.6,4.6,1020.8,0.7,5.0,3,1,2,2014,0,0,0


To overcome the missing data, a study has been conducted to measure the accuracy of different imputation strategies.

After an extended review of previous used imputation models, two different models were chosen:
- Multivariate imputation by chained equations - [MICE](https://github.com/eltonlaw/impyute);
- Miss Forest - [MF](https://pypi.org/project/missingpy/).


Both models impute values multiple times, which is known as multiple imputation technique. Altought it is more computational expensive it outperforms in most case scenarios the single imputation technique.

<img src="Images/Data_Imputation_Study_Diagram.PNG" width="500" height="200" align="center"/>
<center> Fig.1 - Data Imputation Study - Diagram <center>

To conduct this study, the process represented in Fig.1 was performed. This process was done separately for **ECD** and **WCD**, since both models are ruled by the multivariate approach which may lead to an exchange of information between each data set type and a consequent data leakage. This process can be broken down into five main steps:

1. From the entire data set, a selection of the maximum consecutive missing values of each feature was performed, denoted as **target** (Tab.1) This selection was motivated by the increased difficulty of imputing wide ranges of missing values;


2. The data from the year of **2014** was chosen, since it exhibits the lowest number of missing values. The missing values from the adopted year were dropped and the rest of the data was used as a baseline data set to test the different imputation models;


3. A random generation of artificial missing data was performed for each feature based on the **target** (Tab.1) in the baseline data set. Regardless the randomness of this part of the process, when the **WCD** was evaluated, the gaps (missing values) were generated in parallel, in attempt to replicate the real situation;


4. Both models (**MICE** and **MF**), were separately applied, filling the missing values with the correspondent predictions;


5. The imputed values were then compared with the real ones, stored in step 3. The evaluation of the models per feature was conducted using several error metrics. Afterwards, the baseline data set predictions were set back to the real values;



6. **Steps 3 to 4** were repeated for **10 cycles** to ensure that the models performed the imputation in different times of the year. At the end, the **mean error** was calculated per feature to evaluate the imputation models used.

**Steps 1 and 2**

In [4]:
# Steps 1 and 2

# Target Calculation
def consecutive_nan(data, important=True):
    '''
    Function to call
    INPUT: Data; 
           important is set to True it will show the max consecutive NaN for all the features, 
           otherwise only the important ones with missing values.
    OUTPUT: Dictionary with the maximum missing values per feature
    '''
    columns = data.columns
    list_building = []
    list_columns = []
    for i in range(len(columns)):
        list_building.append(consecutive_true(data, columns[i]))
        list_columns.append(columns[i])
        
    missing_values = dict(zip(list_columns, list_building))

    if important==True:
        missing_values = clean_target(missing_values)
    
    return missing_values

def consecutive_true(data, building):
    data = data[building].isna()
    longest = 0
    current = 0
    for num in data:
        if num == True:
            current += 1
        else:
            longest = max(longest, current)
            current = 0

    return max(longest, current)

def clean_target(target):
    clean_target = list() 

    # Iterate over the dict and delete keys in the list
    for (key, value) in target.items() :
        if value == 0:
            clean_target.append(key)

    # Iterate over the list and delete corresponding key from dictionary
    for key in clean_target:
        if key in target:
            del target[key]
    return target

target = consecutive_nan(dt)

In [5]:
target

{'civil': 148,
 'south_tower': 88,
 'wt_temp': 1165,
 'wt_tmpap': 1165,
 'wt_hr': 1165,
 'wt_max_windgust': 1165,
 'wt_mean_windspd': 1165,
 'wt_mean_pres': 1165,
 'wt_mean_solarrad': 1165,
 'wt_rain_day': 1165}

**Steps 3, 4, 5 and 6**

In [6]:
def calculate_mean_errors(target, number_samples, buildings):
    '''
    INPUT:
        target - dict - previous calculated through consecutive_nan function
        number_samples - int - number of cycles to impute - higher number - leads to a more solid evaluation
        buildings  - bool - True: buildings imputation 
                            False: weather conditions imputation
                         
    '''
    # Creation of the matrix
    if buildings == True:
        target = dict(list(target.items())[:2])
    else:
        target = dict(list(target.items())[-8:])
        
    sum_matrix = np.zeros((len(target), 4))
   
    RF_mean_error = imputation(sum_matrix,target, number_samples, buildings, model_name='RF')
    MICE_mean_error = imputation(sum_matrix,target, number_samples, buildings, model_name='MICE')
    MEAN_mean_error = imputation(sum_matrix, target, number_samples, buildings, model_name='MEAN')
 
    return RF_mean_error, MICE_mean_error, MEAN_mean_error

In [19]:
def imputation(sum_matrix, target, number_samples, buildings, model_name):
    '''
    INPUT:
        sum_matrix - zero matrix to add errors of imputation per number_samples
        target - dict - with the maximum consecutive missing values of each column
        number_samples - Int - number of cycles to test
        buildings - Bool - True: buildings imputation 
                           False: weather conditions imputation
        model_name - Str - 'RF': MissForest Model
                           'MICE': Multiple Imputation by Chained Equation Model
                           'MEAN': Uses the mean of the available years at the same hour and month
                       
    OUTPUT:
        mean_matrix_dt - pd.DataFrame - Mean errors of imputed features       
    
    '''
       
    # Starting the training
    print(20*'-'+f' STARTING {model_name} IMPUTATION '+20*'-')
    print('\n')
    
    cycle=0
    
    # Iterate over a random missing value position
    for n in range(number_samples):

        # Refresh DATA - 2014 only - year with less missing data
        # Dataframe of 2017 and 2018 for MEAN_imputation model (model_name = 'MEAN')
        dt = pd.read_csv('Preprocessed_Data/_01_dt_00.csv', index_col='timestamp', parse_dates=[0], header=0)
        dt_ = dt.copy()
        dt = dt['2014'].dropna()
        dt_2017_18 = dt_[dt_.index.year.isin(['2017', '2018'])]
        
        # Evaluation of Buildings or Weather Conditions
        if buildings == True:
            features = [c for c in list(dt.columns) if c == 'civil' or c =='south_tower' or c.startswith('t_')]            
        else:
            features = [c for c in list(dt.columns) if c.startswith('wt_') or c.startswith('t_')]
            
        dt = dt[features]
        dt_2017_18 = dt_2017_18[features]

        # Create random missing values
        print('Random Artificial Missing Values Creation')
        col_create_NaN_target(dt, target, n)

        # Apply models
        if model_name == 'RF':
            imputation = RF_imputation(dt)
            
        elif model_name == 'MICE':
            imputation = MICE_imputation(dt)
            #sum_matrix = MICE_imputation(sum_matrix, dt, target, model_name, n)
            
        elif model_name == 'MEAN':
            imputation = MEAN_imputation(dt, dt_2017_18)
            #sum_matrix = MEAN_imputation(sum_matrix, dt, dt_2017_18, target, model_name, n)

    
        # Calculate Error per column
        imputation_error = error_per_column(dt, imputation, target, model_name, n)

        # Add to a common matrix
        sum_matrix += imputation_error.values[:, 1:].astype('float')
        
        # End of one cycle
        cycle += 1
        print(20*"-"+" "+str(cycle)+"/10 "+"Cycle finished "+20*"-")

    # Calculate mean    
    mean_matrix = sum_matrix/number_samples
    mean_matrix_dt = pd.DataFrame(data=mean_matrix, index=array(dt.columns)[:len(target)],columns=['CV(%)','RMSE', 'MAPE','R_square'])
    
    # Save Dataframe
    if buildings == True:
        mean_matrix_dt.to_csv('01-01-Imputation_Data/'+model_name+'/mean_error/buildings_mean_with_'+str(number_samples)+'.csv')
    else:
        mean_matrix_dt.to_csv('01-01-Imputation_Data/'+model_name+'/mean_error/wt_mean_with_'+str(number_samples)+'.csv')
    
    print(mean_matrix_dt)
    print('\n')
    return mean_matrix_dt

def RF_imputation(dt):
    '''
    INPUT:
        dt - DataFrame for imputation
    OUTPUT:
        rf_imputation - DataFrame imputed
    '''
    
    # Create model
    rf_imputer = MissForest(max_iter=10, n_estimators=100)
    
    # Fit and Transform model
    rf_imputation = rf_imputer.fit_transform(dt)
    rf_imputation = pd.DataFrame(data=rf_imputation,index=dt.index,columns=dt.columns)
    
    return rf_imputation

def MICE_imputation(dt):
    '''
    INPUT:
        dt - DataFrame for imputation
    OUTPUT:
        mice_imputation - DataFrame imputed
    '''
    # Train and Fit
    mice_imputer = mice(dt.values)
    
    # dataframe and merge
    mice_imputation = pd.DataFrame(data=mice_imputer,index=dt.index,columns=dt.columns)
    
    return mice_imputation

def MEAN_imputation(dt, dt_2017_18):
    '''
    INPUT:
        dt - DataFrame for imputation
        dt_2017_18 - To be used to catch mean values
    OUTPUT:
        mean_imputation - DataFrame imputed
    '''
    
    # Unique years
    years = dt.index.year.unique()
    
    # Columns with missing data
    missing_columns = dt.columns[dt.isnull().any()]
    # Iterate over each column with missing data
    for column_name in missing_columns:
        
        for i in range(len(dt)):
            column = dt[column_name][i]
            idx = dt.index[i]      
            hour = idx.hour
            month = idx.month
            year = idx.year

            if math.isnan(column) == True:

                dt_month = dt_2017_18.loc[dt_2017_18['t_month'] == month]
                dt_hour = dt_month.loc[dt_month['t_hour'] == hour]
                impute = dt_hour[column_name].mean()
        
                dt[column_name][i] = impute               
    return dt

In [20]:
'''
Extra functions:
- To artificial create missing values (col_create_NaN_target() + create_NaN_target())
- To measure the error (error_per_column() + calculate_error())

''' 
def col_create_NaN_target(data, target, n):
    '''
    Creates artificial random missing values
    alongside with create_NaN_target function
    '''
    
    building, missing = zip(*target.items())

    for i in range(len(target)):
        data = create_NaN_target(data, building[i], missing[i], n)
        
def create_NaN_target(data, column, length, n):
    
    # Because weather missing values are in the same position
    same_starting = ['wt_temp', 'wt_tmpap','wt_hr', 'wt_max_windgust',
                     'wt_mean_windspd', 'wt_mean_pres','wt_mean_solarrad', 'wt_rain_day']
    
    if column in same_starting:
        random.seed(n)
        num = randint(0, len(data)-length)
        
    elif column == 'civil':
        random.seed(n)
        num = randint(0, len(data)-length)
    elif column == 'south_tower':
        random.seed(n+1)
        num = randint(0, len(data)-length)
    
    # Creation of lists
    list_index=[]
    list_test=[]
    
    # For loop to remove data
    for i in range(len(data)):
        if i in range(num, num+length):
            
            list_test.append(data[column][i])
            list_index.append(data.index[i])
            data[column][i] = np.nan
    
    # Creation of a test dataframe
    test = pd.DataFrame(data=list_test, index=list_index, columns=[column+'_true'])
    
    # Save to check the accuracy later
    filename = '01-01-Imputation_Data/'+column+'_true.csv'
    test.to_csv(filename)
    
    return data

def error_per_column(dt, dt_pred, target, model_used, n):
    
    building, missing = zip(*target.items())
    error = []
    
    for i in range(len(target)):
        filename = '01-01-Imputation_Data/'+building[i]+'_true.csv'
    
        dt_true = pd.read_csv(filename,index_col=[0],parse_dates=[0], header=0)
        dt_both = pd.merge(dt_true, dt_pred[building[i]], how = 'inner', left_index=True, right_index=True)
        
        # Save dataframe to check later
        filename = '01-01-Imputation_Data/'+model_used+'/'+building[i]+'_'+str(n)+'.csv'
        dt_both.to_csv(filename)
    
        # Calculate error and append
        error_column = calculate_error(dt_both)
        error_column.insert(0, building[i]) 
        # Append to a list of lists
        error.append(error_column)
    
    
    error = pd.DataFrame(data=error, columns=['building','CV(%)','RMSE', 'MAPE','R^2'])
    return error

def calculate_error(to_check):
    
    arr = to_check.values
    y_true = arr[:,0]
    y_pred = arr[:,1]
       
    cv = (math.sqrt(mean_squared_error(y_true, y_pred))/mean(y_true))*100
    rmse = math.sqrt(mean_squared_error(y_true, y_pred))
    mape = tsmetrics.mean_absolute_percentage_error(y_true, y_pred, min_val=0.001)
    r2 = r2_score(y_true, y_pred)
    
    error_column = [cv, rmse, mape, r2]

    return error_column

### 1.3.1 Imputation & Results

**BUILDINGS**

In [9]:
RF_mean_error_b, MICE_mean_error_b, MEAN_mean_error_b  = calculate_mean_errors(target, 10, buildings=True)

-------------------- STARTING RF IMPUTATION --------------------


Random Artificial Missing Values Creation
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
-------------------- 1/10 Cycle finished --------------------
Random Artificial Missing Values Creation
Iteration: 0
Iteration: 1
Iteration: 2
-------------------- 2/10 Cycle finished --------------------
Random Artificial Missing Values Creation
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
-------------------- 3/10 Cycle finished --------------------
Random Artificial Missing Values Creation
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
-------------------- 4/10 Cycle finished --------------------
Random Artificial Missing Values Creation
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
-------------------- 5/10 Cycle finished --------------------
Random Artificial Missing Values Creation
Iteration: 0
Iteration: 1
Iteration: 2
-------------------- 6/10 Cycle f

**WEATHER CONDITIONS**

In [10]:
RF_mean_error_wt, MICE_mean_error_wt, MEAN_mean_error_wt  = calculate_mean_errors(target, 10, buildings=False)

-------------------- STARTING RF IMPUTATION --------------------


Random Artificial Missing Values Creation
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
-------------------- 1/10 Cycle finished --------------------
Random Artificial Missing Values Creation
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
-------------------- 2/10 Cycle finished --------------------
Random Artificial Missing Values Creation
Iteration: 0
Iteration: 1
Iteration: 2
-------------------- 3/10 Cycle finished --------------------
Random Artificial Missing Values Creation
Iteration: 0
Iteration: 1
Iteration: 2
-------------------- 4/10 Cycle finished --------------------
Random Artificial Missing Values Creation
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
-------------------- 5/10 Cycle finished --------------------
Random Artificial Missing Values Creation
Iteration: 0
Iterati

After this study we can conclude that although the **Miss Forest** (RF) model showed, in general, greater accuracy in each of the data set types (WCD and ECD) when compared with **MICE**, the **gap in WCD** was still with the undesirable imputation values in terms of the expected seasonality and trend.


To address that, the third method **MEAN_imputation** was developed. This method basically fills each feature independently with the known values from the available years, using the correspondent mean of the same month and hour.


---

## 1.4 Data Imputation Strategy - Creation of different datasets

The last step of **Data Treatment** consists in the creation of three different dataframes that differ in the adopted strategies for the existing missing values.

Therefore, the first strategy common to each of the data sets, named as `dt_01`, `dt_02`, and `dt_03`, was a **three hours linear interpolation**, which is of great use in time series data. After that, the three dataframes differ from one to another, as it is described below:

* `dt_01`
    - Drop every NaN values
* `dt_02`
    - ECD Imputation by **Miss Forest** 
    - Drop WCD NaN values
* `dt_03`
    - ECD Imputation by **Miss Forest**
    - WCD Imputation by **MEAN_Imputation**

In [7]:
dt = pd.read_csv('Preprocessed_Data/_01_dt_00.csv',index_col=[0],parse_dates=[0], header=0)
dt.head()

,civil,south_tower,wt_temp,wt_tmpap,wt_hr,wt_max_windgust,wt_mean_windspd,wt_mean_pres,wt_mean_solarrad,wt_rain_day,t_hour,t_month,t_dayofweek,t_year,miss_civil,miss_south_tower,miss_wt
timestamp,,,,,,,,,,,,,,,,,
2014-01-01 01:00:00,65.99413,114.897996,12.6,10.5,88.3,7.3,3.4,1021.0,0.6,4.9,1,1,2,2014,0,0,0
2014-01-01 02:00:00,66.20412,114.819072,12.9,10.3,87.3,7.8,4.0,1021.0,0.7,4.9,2,1,2,2014,0,0,0
2014-01-01 03:00:00,67.27316,114.491295,13.1,10.2,86.2,8.6,4.6,1020.8,0.7,5.0,3,1,2,2014,0,0,0
2014-01-01 04:00:00,65.53597,115.337955,13.2,10.0,85.9,9.3,4.9,1020.0,0.8,5.0,4,1,2,2014,0,0,0
2014-01-01 05:00:00,68.03676,114.201319,13.2,10.2,85.2,8.6,4.6,1020.0,0.6,5.0,5,1,2,2014,0,0,0


In [8]:
dt.isna().sum()

civil                157
south_tower           96
wt_temp             2357
wt_tmpap            2357
wt_hr               2357
wt_max_windgust     2357
wt_mean_windspd     2357
wt_mean_pres        2357
wt_mean_solarrad    2357
wt_rain_day         2357
t_hour                 0
t_month                0
t_dayofweek            0
t_year                 0
miss_civil             0
miss_south_tower       0
miss_wt                0
dtype: int64

#### dt_01 : Linear Interpolation + Drop every NaN

In [10]:
dt_01 = dt.interpolate(method='linear', limit=3).dropna()

In [12]:
dt_01.to_csv('Preprocessed_Data/_01_dt_01.csv')

#### dt_02 : Linear Interpolation + ECD Miss Forest + Drop WCD NaN

In [13]:
# Columns selection
def select_columns(df, prefix_catch):
    list_col = []   
    for prefix in prefix_catch:
        cols = select_by_prefix(df, prefix)
        list_col.extend(cols)
    return list_col

def select_by_prefix(dt, d_type):
    return list(dt.filter(regex='^'+d_type, axis=1).columns)

In [14]:
def MissForest_buildings(dt):
    ## Split the dataframes to apply multi-imputation

    ## Buildings
    dt_buildings = dt[['civil', 'south_tower']]

    ## Weather and other features
    dt_weather = dt[select_columns(dt,['wt_','miss','t_'])]

    ## BUILDINGS
    mf = MissForest(max_iter=10, n_estimators=100)
    # Fit and Fill the dataframe
    dt_buildings_mf = mf.fit_transform(dt_buildings)
    # Create the dataframe again with the Fill Data
    dt_buildings_mf = pd.DataFrame(data=dt_buildings_mf,index=dt_buildings.index,columns=dt_buildings.columns)


    # Merge the Weather and other features and Buildings Columns
    dt = pd.merge(dt_buildings_mf,dt_weather, how='inner', left_index=True, right_index=True)
    
    return dt

In [15]:
# Linear Interpolation
dt_02 = dt.interpolate(method='linear', limit=3)
# Buildings Imputation
dt_02 = MissForest_buildings(dt_02)
# Drop weather values
dt_02 = dt_02.dropna()

Iteration: 0
Iteration: 1
Iteration: 2


In [24]:
dt_02.to_csv('Preprocessed_Data/_01_dt_02.csv')

#### dt_03 : Linear Interpolation + ECD Miss Forest + WCD MEAN_Imputation

In [17]:
# Linear Interpolation
dt_03 = dt.interpolate(method='linear', limit=3)

# ECD Imputation
dt_03 = MissForest_buildings(dt_03)

# WCD Imputation with MEAN_imputation
dt_no_missing = dt_03.loc[dt_03['miss_wt'] == 0]
dt_03 = MEAN_imputation(dt_03, dt_no_missing)

Iteration: 0
Iteration: 1
Iteration: 2


In [22]:
dt_03.to_csv('Preprocessed_Data/_01_dt_03.csv')

#### Summary of the imputation techniques applied per data frame

<img src="Images/summary_imputation.PNG" width="900" height="200" align="center"/>
<center> Tab.1 -Summary of the imputation techniques applied per data set <center>